In [1]:
!pip install --user ipywidgets
!pip install ydata-profiling
!pip install pandas


[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: Can not perform a '--user' install. User site-packages are not visible in this virtualenv.



[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import numpy as np
import pandas as pd
from ydata_profiling import ProfileReport

In [39]:
df_museos = pd.read_csv("data/201132-0-museos.csv" , encoding='latin-1', sep=';')


In [ ]:
profile = ProfileReport(df_museos, title ="Perfilado museos")
profile.to_notebook_iframe()
profile.to_file("museos.html")

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [7]:
df_museos.describe()

,PK,NUM,PUERTA,ESCALERAS,CODIGO-POSTAL,COD-BARRIO,COD-DISTRITO,COORDENADA-X,COORDENADA-Y,LATITUD,LONGITUD
count,6.900000e+01,69.000000,0.0,0.0,69.000000,69.000000,69.000000,69.000000,6.900000e+01,69.000000,69.000000
mean,2.590645e+06,23.401449,NaN,NaN,28018.000000,3.594203,4.434783,440501.623188,4.475419e+06,40.427288,-3.701564
std,6.668118e+06,39.889953,NaN,NaN,14.279768,1.935115,4.114102,2258.834487,2.894610e+03,0.026067,0.027199
min,2.050000e+03,0.000000,NaN,NaN,28001.000000,1.000000,1.000000,432238.000000,4.469498e+06,40.373620,-3.808430
25%,2.073000e+03,3.000000,NaN,NaN,28008.000000,2.000000,1.000000,439691.000000,4.474015e+06,40.414615,-3.710816
50%,2.114000e+03,8.000000,NaN,NaN,28013.000000,4.000000,3.000000,440627.000000,4.474839e+06,40.421994,-3.700169
75%,4.707297e+06,23.000000,NaN,NaN,28024.000000,5.000000,7.000000,441375.000000,4.476460e+06,40.436756,-3.691157
max,5.006741e+07,237.000000,NaN,NaN,28049.000000,8.000000,21.000000,449698.000000,4.488524e+06,40.545379,-3.593276


In [ ]:
# Filtrar filas fuera del rango [-3.834, -3.566]:
df_fuera_de_madrid = df_museos[(df_museos['LONGITUD'] < -3.8891) | (df_museos['LONGITUD'] > -3.517915)]
# Mostrar una vista previa:
df_fuera_de_madrid.head()

# Filtrar filas fuera del rango [-3.834, -3.566]:
df_fuera_de_madrid2 = df_museos[(df_museos['LATITUD'] < 40.312085) | (df_museos['LATITUD'] > 40.567574)]
# Mostrar una vista previa:
df_fuera_de_madrid2.head()


,PK,NOMBRE,DESCRIPCION-ENTIDAD,HORARIO,EQUIPAMIENTO,TRANSPORTE,DESCRIPCION,ACCESIBILIDAD,CONTENT-URL,NOMBRE-VIA,...,COD-DISTRITO,DISTRITO,COORDENADA-X,COORDENADA-Y,LATITUD,LONGITUD,TELEFONO,FAX,EMAIL,TIPO


In [19]:
# Validar el campo EMAIL: al menos 2 caracteres antes y después de la arroba, dominio .com/.es/.org
import re

# Expresión regular (case-insensitive)
patron = re.compile(r'^[\w\.-]{2,}@[A-Za-z0-9\.-]{2,}\.(com|es|org)$', re.IGNORECASE)

def validar_email(email):
    # Clasificar nulos/vacíos como None (nulo)
    if pd.isna(email):
        return None
    s = str(email).strip()
    if s == '':
        return None
    return bool(patron.match(s))

# Aplicar la validación y añadir columnas
if 'EMAIL' in df_museos.columns:
    df_museos['email_valido'] = df_museos['EMAIL'].apply(validar_email)

    def estado_from_valid(v):
        if pd.isna(v):
            return 'nulo'
        if v:
            return 'valido'
        return 'erroneo'

    df_museos['email_estado'] = df_museos['email_valido'].apply(estado_from_valid)

    emails_erroneos = df_museos[df_museos['email_estado'] == 'erroneo'].copy()
    emails_nulos = df_museos[df_museos['email_estado'] == 'nulo'].copy()

    print(f"Total filas: {len(df_museos)}")
    print(f"Emails válidos: {(df_museos['email_estado'] == 'valido').sum()}")
    print(f"Emails erróneos: {len(emails_erroneos)}")
    print(f"Emails nulos: {len(emails_nulos)}")

    display(emails_erroneos.head(20))
    # Opcional: mostrar nulos
    # display(emails_nulos.head(20))
else:
    print("La columna 'EMAIL' no existe en el DataFrame.")


Total filas: 69
Emails válidos: 50
Emails erróneos: 3
Emails nulos: 16


,PK,NOMBRE,DESCRIPCION-ENTIDAD,HORARIO,EQUIPAMIENTO,TRANSPORTE,DESCRIPCION,ACCESIBILIDAD,CONTENT-URL,NOMBRE-VIA,...,COORDENADA-X,COORDENADA-Y,LATITUD,LONGITUD,TELEFONO,FAX,EMAIL,TIPO,email_valido,email_estado
5,2069,Centro Cultural La Corrala. Museo de Artes y T...,NaN,Lunes a viernes de 10 a 20 horas. Sábados de 1...,Las visitas individuales (grupos inferiores a ...,Metro: La Latina (línea 5),NaN,3,http://www.madrid.es/sites/v/index.jsp?vgnextc...,CARLOS ARNICHES,...,439899,4473375,40.408827,-3.708324,914 976 500,NaN,museoatp@uam.es / lacorrala@uam.es,/contenido/entidadesYorganismos/Museos,False,erroneo
38,2092,Museo de Astronomía y Geodesia,El museo de Astronomía y Geodesia de la Univer...,Visita con cita previa.,NaN,Metro: Ciudad Universitaria (línea 6).,NaN,3,http://www.madrid.es/sites/v/index.jsp?vgnextc...,CIENCIAS,...,438442,4477897,40.449459,-3.725938,91 394 4588,NaN,fuensant@ucm.es.,/contenido/entidadesYorganismos/Museos,False,erroneo
41,2084,Museo de Cera,NaN,"Horario de invierno: Lunes a domingo, de 11 a ...",NaN,Metro: Colón (línea 4),NaN,2,http://www.madrid.es/sites/v/index.jsp?vgnextc...,RECOLETOS,...,441350,4475102,40.424488,-3.691381,913 199 330,NaN,info@museoceramadrid.com; reservas@museocerama...,/contenido/entidadesYorganismos/Museos,False,erroneo


In [21]:
# Validar el campo TELEFONO: deben ser 9 dígitos y empezar por '91'.
import re

def validar_telefono(tel):
    # Clasificar nulos/vacíos como None
    if pd.isna(tel):
        return None
    s = str(tel).strip()
    if s == '':
        return None
    # Mantener solo dígitos
    digits = re.sub(r"\D", "", s)
    if digits == '':
        return None
    # Validación: exactamente 9 dígitos y empieza por '91'
    if len(digits) == 9 and digits.startswith('91'):
        return True
    return False

# Aplicar la validación y añadir columnas
if 'TELEFONO' in df_museos.columns:
    df_museos['telefono_valido'] = df_museos['TELEFONO'].apply(validar_telefono)

    def estado_tel(v):
        if pd.isna(v):
            return 'nulo'
        if v:
            return 'valido'
        return 'erroneo'

    df_museos['telefono_estado'] = df_museos['telefono_valido'].apply(estado_tel)

    telefonos_erroneos = df_museos[df_museos['telefono_estado'] == 'erroneo'].copy()
    telefonos_nulos = df_museos[df_museos['telefono_estado'] == 'nulo'].copy()

    print(f"Total filas: {len(df_museos)}")
    print(f"Teléfonos válidos: {(df_museos['telefono_estado'] == 'valido').sum()}")
    print(f"Teléfonos erróneos: {len(telefonos_erroneos)}")
    print(f"Teléfonos nulos: {len(telefonos_nulos)}")

    display(telefonos_erroneos.head(20))
else:
    print("La columna 'TELEFONO' no existe en el DataFrame.")


Total filas: 69
Teléfonos válidos: 44
Teléfonos erróneos: 13
Teléfonos nulos: 12


,PK,NOMBRE,DESCRIPCION-ENTIDAD,HORARIO,EQUIPAMIENTO,TRANSPORTE,DESCRIPCION,ACCESIBILIDAD,CONTENT-URL,NOMBRE-VIA,...,LATITUD,LONGITUD,TELEFONO,FAX,EMAIL,TIPO,email_valido,email_estado,telefono_valido,telefono_estado
6,6469186,Ermita de San Antonio de la Florida (Museo),Ermita dedicada a San Antonio de Padua y peque...,General De martes a domingos de 9:30 a 20 ho...,"Tienda, en la que se pueden adquirir las guías...","Metro: Príncipe Pío (líneas 5, 10 y ramal Óper...",NaN,"2,6",http://www.madrid.es/sites/v/index.jsp?vgnextc...,SAN ANTONIO DE LA FLORIDA,...,40.425257,-3.725726,915 420 722 o 683 429 760,NaN,sanantonio@madrid.es,/contenido/entidadesYorganismos/MonumentosEdif...,True,valido,False,erroneo
7,58353,Espacio Fundación Telefónica,NaN,"Martes, miércoles, jueves y viernes de 11 a 20...",NaN,Metro: Gran Vía (líneas 1 y 5). Aparcamientos...,NaN,"1,6",http://www.madrid.es/sites/v/index.jsp?vgnextc...,FUENCARRAL,...,40.420425,-3.701595,914 984 273 (información general) / 679 765 25...,NaN,espacio@fundaciontelefonica.com,/contenido/entidadesYorganismos/Museos,True,valido,False,erroneo
10,79870,Imprenta Municipal - Artes del libro,"La Imprenta Municipal-Artes del Libro, inaugur...",De martes a domingos y festivos: de 10 a 20 ho...,Salón de actos. Biblioteca especializada y Cen...,"Metro: Sol, Tirso de Molina, Latina Bus: 3, 6,...",NaN,"1,6",http://www.madrid.es/sites/v/index.jsp?vgnextc...,CONCEPCION JERONIMA,...,40.413897,-3.705469,Lunes a viernes de 9 a 15 h: 914 294 271\nLune...,NaN,imprentamunicipal@madrid.es,/contenido/entidadesYorganismos/Museos,True,valido,False,erroneo
16,50067411,Museo Banksy Madrid,Nuevo espacio cultural en Madrid,De lunes a domingo de 10 a 20 horas (última en...,Más datos en https://museobanksy.es/museo-bank...,Metro: Acacias (línea 5) / Embajadores (líneas...,NaN,1,http://www.madrid.es/sites/v/index.jsp?vgnextc...,ESPERANZA,...,40.403736,-3.706194,603 607 564,NaN,info.madrid@museobanksy.es,/contenido/entidadesYorganismos/Museos,True,valido,False,erroneo
18,4112,Museo Cerralbo,"Un palacio del siglo XIX, un museo del siglo X...","El museo abre: Martes, miércoles, jueves, v...",NaN,Metro: Ventura Rodríguez (línea 3); Plaza de E...,Tarifas Accesibilidad Visitas en grupo,"2,6",http://www.madrid.es/sites/v/index.jsp?vgnextc...,VENTURA RODRIGUEZ,...,40.423779,-3.714600,Tel.: 915 473 646 / 647. Atención telefónica d...,NaN,museo.cerralbo@cultura.gob.es,/contenido/entidadesYorganismos/BibliotecasPat...,True,valido,False,erroneo
24,6019032,Museo Los Caños del Peral. Estación de Metro Ó...,"A diez metros bajo la plaza de Isabel II, la e...",Previa reserva en la página museosmetromadrid....,NaN,"Metro: Ópera (líneas 2, 5 y Ramal).",NaN,1,http://www.madrid.es/sites/v/index.jsp?vgnextc...,ISABEL II,...,40.417763,-3.709188,644 169 531,NaN,info@museosmetromadrid.es,/contenido/entidadesYorganismos/Museos,True,valido,False,erroneo
28,2057,Museo Nacional de Antropología,El Museo está instalado en el edificio de esti...,De martes a sábado de 9:30 a 20 horas. Domingo...,NaN,Metro: Estación del Arte (línea 1). Cercanías...,NaN,"1,6",http://www.madrid.es/sites/v/index.jsp?vgnextc...,ALFONSO XII,...,40.407894,-3.688858,915 306 418 / 915 395 995,914 677 098,antropologico@cultura.gob.es,/contenido/entidadesYorganismos/Museos,True,valido,False,erroneo
29,2055,Museo Nacional de Artes Decorativas,"Inaugurado en 1912, su sede es un típico palac...",Martes a sábados de 9:30 a 15 horas. Domingo y...,NaN,Metro: Banco de España (Línea 2),El Museo se encuentra inmerso en un proceso de...,"2,6",http://www.madrid.es/sites/v/index.jsp?vgnextc...,MONTALBAN,...,40.417897,-3.689755,915 326 499 / 915 326 845,915 232 086,mnad@cultura.gob.es,/contenido/entidadesYorganismos/MonumentosEdif...,True,valido,False,erroneo
36,2091,Museo de América,NaN,"De martes a sábado, de 9:30 a 15 horas. Jueves...",NaN,"Metro: Moncloa (líneas 3 y 6), Islas Filipinas...",NaN,"1,6",http://www.madrid.es/sites/v/index.jsp?vgnextc...,REYES CATOLICOS,...,40.437937,-3.72

In [46]:
# Rellenar emails vacíos/nulos con contactanos@madrid.es
df_museos['EMAIL'] = df_museos['EMAIL'].fillna('contactanos@madrid.es')
df_museos['EMAIL'] = df_museos['EMAIL'].replace('', 'contactanos@madrid.es')

# Re-validar los emails después del relleno
df_museos['email_valido'] = df_museos['EMAIL'].apply(validar_email)

def estado_from_valid(v):
    if pd.isna(v):
        return 'nulo'
    if v:
        return 'valido'
    return 'erroneo'

df_museos['email_estado'] = df_museos['email_valido'].apply(estado_from_valid)

# Corregir emails erróneos: eliminar todo después de .es/.com/.org
def limpiar_email_erroneo(email):
    """Elimina todo lo que aparezca después de .es, .com o .org"""
    if pd.isna(email):
        return email
    s = str(email).strip()
    # Buscar las extensiones válidas y eliminar todo después
    for ext in ['.es', '.com', '.org']:
        if ext in s:
            # Encontrar la posición y tomar todo hasta ext (inclusive)
            pos = s.find(ext)
            return s[:pos + len(ext)]
    return s

# Aplicar la limpieza solo a emails erróneos
emails_con_error = df_museos['email_estado'] == 'erroneo'
df_museos.loc[emails_con_error, 'EMAIL'] = df_museos.loc[emails_con_error, 'EMAIL'].apply(limpiar_email_erroneo)

# Re-validar los emails después de la limpieza
df_museos['email_valido'] = df_museos['EMAIL'].apply(validar_email)
df_museos['email_estado'] = df_museos['email_valido'].apply(estado_from_valid)

print(f"Total filas: {len(df_museos)}")
print(f"Emails válidos: {(df_museos['email_estado'] == 'valido').sum()}")
print(f"Emails erróneos: {(df_museos['email_estado'] == 'erroneo').sum()}")
print(f"Emails nulos: {(df_museos['email_estado'] == 'nulo').sum()}")

# Mostrar algunos ejemplos del nuevo email por defecto
print(f"\nEjemplos de filas con email por defecto:")
display(df_museos[df_museos['EMAIL'] == 'contactanos@madrid.es'][['NOMBRE', 'EMAIL']].head(10))

# Mostrar ejemplos de emails corregidos
print(f"\nEjemplos de emails corregidos:")
muestra_corregidos = df_museos[emails_con_error][['NOMBRE', 'EMAIL']].head(10)
display(muestra_corregidos)


Total filas: 69
Emails válidos: 69
Emails erróneos: 0
Emails nulos: 0

Ejemplos de filas con email por defecto:


,NOMBRE,EMAIL
8,Frontón Beti Jai,contactanos@madrid.es
9,Galería de la Colecciones Reales,contactanos@madrid.es
11,La Casa Encendida de la Fundación Montemadrid,contactanos@madrid.es
12,Monasterio de las Descalzas Reales,contactanos@madrid.es
14,Museo Africano Misioneros Combonianos,contactanos@madrid.es
15,Museo Arqueológico Nacional,contactanos@madrid.es
19,Museo EMT,contactanos@madrid.es
30,Museo Nacional de Ciencias Naturales,contactanos@madrid.es
40,Museo de Calcografía Nacional,contactanos@madrid.es
56,Museo del Seguro. Fundación Mapfre,contactanos@madrid.es



Ejemplos de emails corregidos:


,NOMBRE,EMAIL
5,Centro Cultural La Corrala. Museo de Artes y T...,museoatp@uam.es
38,Museo de Astronomía y Geodesia,fuensant@ucm.es
41,Museo de Cera,info@museoceramadrid.com


In [40]:
# Procesar el campo TELEFONO: trimmar, extraer primer número 9 dígitos comenzando por 91
# y guardar información adicional en nuevo campo
import re

def procesar_telefono(tel):
    """
    Extrae la primera ocurrencia de 9 dígitos comenzando por '91'.
    Retorna tuple (telefono_limpio, info_adicional)
    Solo extrae dígitos puros (911234567, etc.)
    """
    if pd.isna(tel):
        return None, None
    
    # Trimmar espacios
    s = str(tel).strip()
    if s == '':
        return None, None
    
    # Extraer SOLO los dígitos del string original
    digitos = re.sub(r'\D', '', s)
    
    # Buscar la primera ocurrencia de 9 dígitos comenzando por 91
    for i in range(len(digitos) - 8):
        substr = digitos[i:i+9]
        if substr.startswith('91'):
            # Encontramos un teléfono válido
            telefono_valido = substr
            
            # Información adicional: el resto de caracteres que no forman parte del teléfono
            digitos_antes = digitos[:i]
            digitos_despues = digitos[i+9:]
            info_adicional = (digitos_antes + ' ' + digitos_despues).strip()
            info_adicional = info_adicional if info_adicional else None
            
            return telefono_valido, info_adicional
    
    # No hay teléfono válido, todo es información adicional
    return None, digitos if digitos else None

# Aplicar el procesamiento
if 'TELEFONO' in df_museos.columns:
    resultado = df_museos['TELEFONO'].apply(procesar_telefono)
    df_museos['TELEFONO'] = resultado.apply(lambda x: x[0])
    df_museos['info_adicional_contacto'] = resultado.apply(lambda x: x[1])
    
    # Rellenar SOLO los teléfonos que quedaron None (nulos) con 914800010
    df_museos.loc[df_museos['TELEFONO'].isna(), 'TELEFONO'] = '914800010'
    
    # Re-validar los teléfonos después del procesamiento
    df_museos['telefono_valido'] = df_museos['TELEFONO'].apply(validar_telefono)
    
    def estado_tel(v):
        if pd.isna(v):
            return 'nulo'
        if v:
            return 'valido'
        return 'erroneo'
    
    df_museos['telefono_estado'] = df_museos['telefono_valido'].apply(estado_tel)
    
    print(f"Total filas: {len(df_museos)}")
    print(f"Teléfonos válidos: {(df_museos['telefono_estado'] == 'valido').sum()}")
    print(f"Teléfonos erróneos: {(df_museos['telefono_estado'] == 'erroneo').sum()}")
    print(f"Teléfonos nulos: {(df_museos['telefono_estado'] == 'nulo').sum()}")
    print(f"Registros con información adicional: {df_museos['info_adicional_contacto'].notna().sum()}")
    
    # Mostrar algunos ejemplos del procesamiento
    print(f"\nEjemplos de registros con información adicional:")
    muestra = df_museos[df_museos['info_adicional_contacto'].notna()][['NOMBRE', 'TELEFONO', 'info_adicional_contacto']].head(10)
    display(muestra)
else:
    print("La columna 'TELEFONO' no existe en el DataFrame.")


Total filas: 69
Teléfonos válidos: 69
Teléfonos erróneos: 0
Teléfonos nulos: 0
Registros con información adicional: 13

Ejemplos de registros con información adicional:


,NOMBRE,TELEFONO,info_adicional_contacto
6,Ermita de San Antonio de la Florida (Museo),915420722,683429760
7,Espacio Fundación Telefónica,914984273,679765254
10,Imprenta Municipal - Artes del libro,915914294,27115209147775491020914777549
16,Museo Banksy Madrid,914800010,603607564
18,Museo Cerralbo,915473646,647914
24,Museo Los Caños del Peral. Estación de Metro Ó...,914800010,644169531
28,Museo Nacional de Antropología,915306418,915395995
29,Museo Nacional de Artes Decorativas,915326499,915326845
36,Museo de América,915492641,915439437
45,Museo de Mineralogía,914975407,914972564914974800


In [47]:
df_museos.head(5)


,PK,NOMBRE,DESCRIPCION-ENTIDAD,HORARIO,EQUIPAMIENTO,TRANSPORTE,DESCRIPCION,ACCESIBILIDAD,CONTENT-URL,NOMBRE-VIA,...,LONGITUD,TELEFONO,FAX,EMAIL,TIPO,info_adicional_contacto,telefono_valido,telefono_estado,email_valido,email_estado
0,6017808,Casa Museo Fuente del Rey,"Fundación AMYC - Fran Daurel Este museo, sit...",De lunes a viernes de 10:00 a 19:00 horas. Vis...,NaN,Consulta los siguientes enlaces:,NaN,3,http://www.madrid.es/sites/v/index.jsp?vgnextc...,FUENTE DEL REY,...,-3.770572,913579116,NaN,info@fundaciofrandaurel.com,/contenido/entidadesYorganismos/FundacionesCul...,None,True,valido,True,valido
1,4847190,Casa Museo Lope de Vega,"'Mi casilla, mi quietud, mi g&amp;uuml;ertecil...","De martes a domingo, de 10 a 18 horas. Cerrad...",NaN,"Metro: Antón Martín (línea 1), Sevilla (línea ...",NaN,"0,4",http://www.madrid.es/sites/v/index.jsp?vgnextc...,CERVANTES,...,-3.697467,914299216,NaN,casamuseolopedevega@madrid.org,/contenido/entidadesYorganismos/Museos,None,True,valido,True,valido
2,4949641,Casita Museo de Ratón Pérez,La Casita-Museo constituye un espacio singular...,Visitas: horarios y reservas.,NaN,"Metro: Sol (líneas: 1, 2 y 3), Ópera (líneas 2...",Se aconseja acudan con la entrada adquirida an...,0,http://www.madrid.es/sites/v/index.jsp?vgnextc...,ARENAL,...,-3.705520,915226968,NaN,info@casamuseoratonperez.es,/contenido/entidadesYorganismos/Museos,None,True,valido,True,valido
3,5682031,Castillo de la Alameda,El Castillo de la Alameda o de Barajas es uno ...,Abierto solo en fin de semana (sábado y doming...,Zona wifi.,Metro: Alameda de Osuna y El Capricho (línea 5).,Accesibilidad Rampas de acceso (las rampas d...,"1,6",http://www.madrid.es/sites/v/index.jsp?vgnextc...,ANTONIO SANCHA,...,-3.593276,913667415,NaN,castilloalameda@madrid.es,/contenido/entidadesYorganismos/MonumentosEdif...,None,True,valido,True,valido
4,2064,Casón del Buen Retiro,El Centro de Estudios del Prado está ubicado e...,NaN,NaN,"Metro: Estación del Arte (línea 1), Banco de E...",Descripción monumental El Casón o Salón de Ba...,1,http://www.madrid.es/sites/v/index.jsp?vgnextc...,ALFONSO XII,...,-3.689222,913302821,NaN,centro.estudios@museodelprado.es,/contenido/entidadesYorganismos/MonumentosEdif...,None,True,valido,True,valido


In [ ]:
# Dropear las últimas 4 columnas y exportar a CSV
df_museos_final = df_museos.drop(df_museos.columns[-4:], axis=1)

# Exportar a CSV en la carpeta data
df_museos_final.to_csv('data/museos_corregido.csv', index=False, encoding='latin-1', sep=';')

print(f"Columnas originales: {len(df_museos.columns)}")
print(f"Columnas finales: {len(df_museos_final.columns)}")
print(f"Filas: {len(df_museos_final)}")
print(f"\nArchivo exportado a: data/museos_corregido.csv")
print(f"\nÚltimas columnas eliminadas:")
print(df_museos.columns[-4:].tolist())
print(f"\nColumnas finales:")
print(df_museos_final.columns.tolist())


In [4]:
import sqlite3

In [5]:


conn = sqlite3.connect('my_database.db')
cursor = conn.cursor()

In [6]:
print(df.head())

   Rank                                         book title  book price  \
0     1                       Iron Flame (The Empyrean, 2)       18.42   
1     2                                    The Woman in Me       20.93   
2     3                                  My Name Is Barbra       31.50   
3     4  Friends, Lovers, and the Big Terrible Thing: A...       23.99   
4     5                              How to Catch a Turkey        5.65   

   rating            author  year of publication               genre  \
0     4.1    Rebecca Yarros                 2023     Fantasy Romance   
1     4.5    Britney Spears                 2023              Memoir   
2     4.5  Barbra Streisand                 2023       Autobiography   
3     4.4     Matthew Perry                 2023              Memoir   
4     4.8      Adam Wallace                 2018  Childrens, Fiction   

                                                 url  
0  amazon.com/Iron-Flame-Empyrean-Rebecca-Yarros/...  
1  amazon.co

In [7]:
cursor.execute('''
create table if not exists books (
    Rank real,
    book_title text,
    book_price real,
    rating real,
    author text,
    year_of_publication real,
    genre text,
    url text
)
''')
conn.commit()

In [8]:

with sqlite3.connect('my_database.db') as conn:
    df.to_sql('books', conn, if_exists='replace', index=False)

    print("Data inserted successfully!")


Data inserted successfully!


In [10]:
with sqlite3.connect('my_database.db') as conn:
    query = "SELECT * FROM books LIMIT 100"  
    result = pd.read_sql_query(query, conn)
    print(result)

    Rank                                         book title  book price  \
0      1                       Iron Flame (The Empyrean, 2)       18.42   
1      2                                    The Woman in Me       20.93   
2      3                                  My Name Is Barbra       31.50   
3      4  Friends, Lovers, and the Big Terrible Thing: A...       23.99   
4      5                              How to Catch a Turkey        5.65   
..   ...                                                ...         ...   
95    96  First Little Readers Parent Pack: Guided Readi...       11.40   
96    97                                            Hatchet        5.14   
97    98  The Wager: A Tale of Shipwreck, Mutiny and Murder       15.30   
98    99  I'm Dead, Now What?: Important Information Abo...       12.99   
99   100                                   The Wonky Donkey        5.28   

    rating              author  year of publication  \
0      4.1      Rebecca Yarros              

In [11]:
with sqlite3.connect('my_database.db') as conn:
    query = "SELECT * FROM books"
    df2 = pd.read_sql_query(query,conn)
    df2.to_csv('resultado.csv', index=False)